In [ ]:
path = "/content/drive/MyDrive/Video_grouping_results/VideoGroupingProject"


In [ ]:
!pip install "git+https://github.com/facebookresearch/pytorchvideo.git"


  Cloning https://github.com/facebookresearch/pytorchvideo.git to /tmp/pip-req-build-nc9mmpkf
  Running command git clone -q https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-req-build-nc9mmpkf
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 37.2MB 78kB/s 
     |████████████████████████████████| 645kB 40.5MB/s 
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.1-cp37-none-any.whl size=155238 sha256=832cee96868d246deb9e9b49a011df23ccc997bbe433824137b1eb3b0c48b0bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-zsj2qxef/wheels/56/74/10/f3f9925bf7096f72530ef877ce52dab12f4c00486990bd7e32
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210604-cp37-none-any.whl size=60475 sha256=a61cff8407c4d1c13cd4425b142a4846ec891b79ce4fe04cf7fc048ea911f359
  Stored in directory: /root/.cache/pip/wheels/74/e3/fb/2d4b0c12020f6e4535c2a8fb2da9b6b17cf9f03efd4e542ffc
Successfully built pytorchvideo fvcore
  Found existing installation

In [ ]:
import glob
import os
from collections import defaultdict
import cv2
import math 
import numpy as np
import matplotlib.pyplot as plt
import time

#Pytroch Video

In [ ]:
import torch
import json
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
from typing import Dict


In [ ]:
print(torch.__version__
      )

1.8.1+cu101


In [ ]:
# Device on which to run the model
# Set to cuda to load on GPU
device = "cuda"

# Pick a pretrained model and load the pretrained weights
model_name = "slowfast_r50"
model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=True)

# Set to eval mode and move to desired device
model = model.to(device)
model = model.eval()

Downloading: "https://github.com/facebookresearch/pytorchvideo/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/SLOWFAST_8x8_R50.pyth" to /root/.cache/torch/hub/checkpoints/SLOWFAST_8x8_R50.pyth


In [ ]:
!wget https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json


--2021-06-08 01:35:57--  https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10326 (10K) [text/plain]
Saving to: ‘kinetics_classnames.json’

kinetics_classnames 100%[===================>]  10.08K  --.-KB/s    in 0s      

2021-06-08 01:35:57 (95.9 MB/s) - ‘kinetics_classnames.json’ saved [10326/10326]



In [ ]:
with open("kinetics_classnames.json", "r") as f:
    kinetics_classnames = json.load(f)

# Create an id to label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

In [ ]:
####################
# SlowFast transform
####################

side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
alpha = 4

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list



In [ ]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

In [ ]:
# Load the example video

tagsdict = dict() 

alltagsdict = dict()

for videoname in os.listdir(path)[:1]:

  video_path = path + "/" + videoname
  
  print("VideoName {0} :".format(videoname))

  cap = cv2.VideoCapture(video_path)


  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  #sampling_rate = 2
  frames_per_second = int(cap.get(cv2.CAP_PROP_FPS))

  seconds = int(total_frames / frames_per_second)

  print("Total clip duration ", seconds)

  cap.release()

  num_frames = 32
  sampling_rate = 2
  #frames_per_second = 30

  #print(num_frames ,  frames_per_second)


  # The duration of the input clip is also specific to the model.
  clip_duration = (num_frames * sampling_rate)/frames_per_second

  transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)
  

  videotransform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)


  #defaultdict(list)

  # Select the duration of the clip to load by specifying the start and end duration
  # The start_sec should correspond to where the action occurs in the video
 
  videocompleted = False

  start_sec = 0 
  end_sec = start_sec + clip_duration

  videotags = dict()

  size = (512 , 256)

  result = cv2.VideoWriter("/content/drive/MyDrive/Video_grouping_results/" + videoname , 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         int(frames_per_second/2), size)
    
  all_tagsinvideo = []
  
  while not videocompleted :

    if end_sec > seconds :
      videocompleted = True
      
      end_sec = seconds

    # Initialize an EncodedVideo helper class
    video = EncodedVideo.from_path(video_path)

    start = time.time()

    # Load the desired clip
    video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)



    # Apply a transform to normalize the video input
    try :
      video_data = transform(video_data)

    except: 

      continue

    # Move the inputs to the desired device
    inputs = video_data["video"]
    
    normalten = inputs[1].permute((1,0, 2,3))

    print(normalten.shape)

    un = UnNormalize(mean ,std)

    
      
    inputs = [i.to(device)[None, ...] for i in inputs]

    
    # Pass the input clip through the model
    preds = model(inputs)

    

    # Get the predicted classes
    post_act = torch.nn.Softmax(dim=1)

    end = time.time()

    print("Time taken ",end - start)


    preds = post_act(preds)
    pred_classes = preds.topk(k=10).indices

    # Map the predicted classes to the label names
    pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes[0]]

    print("Clip Duration : {0} with start_sec at {1} and end_sec at {2}".format(clip_duration,start_sec,end_sec))

    print("Predicted labels: %s" % ", ".join(pred_class_names))

    videotags[round(start_sec,2)] = pred_class_names

    for pred_c_n in pred_class_names :

      if pred_c_n not in all_tagsinvideo :
        all_tagsinvideo.append(pred_c_n)

    start_sec += clip_duration 
    end_sec = (start_sec + clip_duration)


    for tensor in normalten:
      
      img = un(tensor)

      img = img.mul(255).byte()

      img = img.cpu().numpy().transpose((1, 2, 0))
    
      # # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
      #image = np.clip(image, 0, 1)

      #print(image.shape)




      image = img[:, :,::-1].astype('uint8')

      image = cv2.copyMakeBorder( image, 0, 0, 256, 0, cv2.BORDER_CONSTANT)

      for i,text in enumerate(pred_class_names):
        image = cv2.putText(image, text, (10, (i+1)*25 ) , cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                 (255,255,255), 1, cv2.LINE_AA)
        
      #print(image.shape)

      # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

      # plt.show() 

      result.write(image)

  print(all_tagsinvideo)

  tagsdict[videoname] = videotags



  result.release()


with open("tags.json", "w") as outfile: 
  json.dump(tagsdict, outfile)




VideoName Video_22.mp4 :
Total clip duration  60
torch.Size([32, 3, 256, 256])
Time taken  0.8932244777679443
Clip Duration : 2.1333333333333333 with start_sec at 0 and end_sec at 2.1333333333333333
Predicted labels: eating burger, dining, eating hotdog, eating doughnuts, celebrating, tasting food, auctioning, eating ice cream, eating cake, playing ukulele
torch.Size([32, 3, 256, 256])
Time taken  0.9100818634033203
Clip Duration : 2.1333333333333333 with start_sec at 2.1333333333333333 and end_sec at 4.266666666666667
Predicted labels: dining, celebrating, eating burger, auctioning, drinking beer, tasting food, tasting beer, eating hotdog, drinking shots, singing
torch.Size([32, 3, 256, 256])
Time taken  0.9158265590667725
Clip Duration : 2.1333333333333333 with start_sec at 4.266666666666667 and end_sec at 6.4
Predicted labels: celebrating, dining, auctioning, pumping fist, eating burger, drinking beer, drinking shots, blowing out candles, singing, bartending
torch.Size([32, 3, 256, 

In [ ]:
!pip install spacy 

In [ ]:
listoftags = ['eating burger', 'dining', 'eating hotdog', 'eating doughnuts', 'celebrating', 'tasting food', 
 'auctioning', 'eating ice cream', 'eating cake', 'playing ukulele', 'drinking beer', 'tasting beer', 
 'drinking shots', 'singing', 'pumping fist', 'blowing out candles', 'bartending', 'dancing macarena', 
 'krumping', 'dancing gangnam style', 'air drumming', 'zumba', 'counting money', 'playing poker', 
 'arm wrestling', 'marching', 'shaking hands', 'waiting in line', 'playing trumpet', 'playing trombone', 
 'riding elephant', 'eating spaghetti', 'kissing', 'hugging', 'eating chips', 'cracking neck', 'sniffing', 
 'peeling apples', 'playing saxophone', 'setting table', 'holding snake', 'surfing crowd', 'country line dancing', 
 'beatboxing', 'playing clarinet', 'tap dancing', 'rock scissors paper', 'playing violin', 'finger snapping', 
 'playing recorder', 'busking', 'clapping', 'dancing charleston', 'swing dancing', 'belly dancing', 'tango dancing', 
 'salsa dancing', 'sign language interpreting', 'contact juggling', 'making pizza', 'making a sandwich', 'barbequing',
 'baking cookies', 'cooking chicken', 'cooking sausages', 'carving pumpkin', 'making a cake', 'capoeira', 'drinking',
 'stomping grapes', 'giving or receiving award', 'braiding hair', 'fixing hair', 'playing accordion', 'shaving head', 
 'getting a haircut', 'riding camel', 
 'laughing', 'knitting', 'curling hair', 'making jewelry', 'smoking', 'smoking hookah', 'tying knot (not on a tie)']


In [ ]:
!git clone https://github.com/PGCodehub/HVU-Dataset.git

fatal: destination path 'HVU-Dataset' already exists and is not an empty directory.


In [ ]:
import pandas as pd

In [ ]:
csv = pd.read_csv("/content/HVU-Dataset/HVU_Tags_Categories_V1.0.csv")

csv.head()

,Tag,Category
0,playing_trombone,action
1,playing_controller,action
2,photograph,action
3,recreation,action
4,sitting,action


In [ ]:
df1 = csv.groupby('Category')['Tag'].apply(list).reset_index(name='Listoftags')

df1.head()

,Category,Listoftags
0,action,"[playing_trombone, playing_controller, photogr..."
1,attribute,"[fun, beauty, leisure, human_hair_color, black..."
2,concept,"[classical_music, music, electronics, technolo..."
3,event,"[summer, night, concert, vacation, wedding, ce..."
4,object,"[brass_instrument, musical_instrument, trumpet..."


In [ ]:
def tokenizer(mylist):
    tokenized_list=[]
    for word in mylist:
        if '_'  in word:
            word = word.replace('_' , ' ').lower()
            tokenized_list.append(word)
      
        else:
            tokenized_list.append(word.lower())
    return(tokenized_list)


In [ ]:
HUV_dataset = dict()

for index, row in df1.iterrows():
    
    HUV_dataset[row['Category']] = tokenizer(row['Listoftags'])





In [ ]:
HUV_dataset.keys()


dict_keys(['action', 'attribute', 'concept', 'event', 'object', 'scene'])

In [ ]:
if 'eating burger' in HUV_dataset['action']:
  print("dsd")

dsd


In [ ]:
import spacy

In [ ]:
#!python -m spacy download en_core_web_lg

In [ ]:
nlp = spacy.load('en_core_web_lg')


In [ ]:
HUV_dataset_nlp = []

for action in HUV_dataset['action']:

  HUV_dataset_nlp.append(nlp(action))


In [ ]:
# compute similarity    
similarities = {}   
for word in listoftags:
    tok = nlp(word)
    similarities[tok.text] ={}
    for tok_ in HUV_dataset_nlp:

        #print(tok_)

        #tok_ = nlp(tok_)
        similarities[tok.text].update({tok_.text:tok.similarity(tok_)})

# sort
top10 = lambda x: {k: v for k, v in sorted(similarities[x].items(), key=lambda item: item[1], reverse=True)[:10]}

In [ ]:
similarities[listoftags[0]]

In [ ]:
top10(listoftags[4])

{'attending conference': 0.4368617391961273,
 'celebrating': 1.0,
 'cheering': 0.42758004722142734,
 'country line dancing': 0.41783343037774123,
 'decorating the christmas tree': 0.4156155066898929,
 'gospel singing in church': 0.4311155152140683,
 'making a cake': 0.3995945569998262,
 'opening present': 0.40964339638496994,
 'visiting the zoo': 0.4185037631464736,
 'wrapping present': 0.3976307342569076}

In [ ]:
listoftags[0]

'eating burger'

In [ ]:
for tag in listoftags[:20]:

  print("Tag "+tag+ " close Words :")
  print(top10(tag))

Tag eating burger close Words :
{'eating burger': 1.0, 'eating hotdog': 0.8811544464160944, 'eating': 0.869451134611863, 'eating doughnuts': 0.8294750236838928, 'eating cake': 0.8262713836216452, 'eating spaghetti': 0.801321227023556, 'eating chips': 0.8006912498677948, 'eating watermelon': 0.7970364289951508, 'eating carrots': 0.7914155290191551, 'tasting food': 0.7191837070256192}
Tag dining close Words :
{'dining': 1.0, 'outdoor recreation': 0.5894881706918668, 'setting table': 0.5638255741853612, 'tasting food': 0.5483881532989527, 'moving furniture': 0.5239409713768117, 'painting furniture': 0.5160987028950328, 'cooking sausages not on barbeque ': 0.5160624759757587, 'home roasting coffee': 0.5079771866205971, 'washing dishes': 0.4997844257994517, 'making sushi': 0.49978076839342145}
Tag eating hotdog close Words :
{'eating hotdog': 1.0, 'eating burger': 0.8811544464160944, 'eating': 0.84430319015075, 'eating doughnuts': 0.8316567836602358, 'eating spaghetti': 0.8032764664890255, 

In [ ]:
HUV_dataset_objects_nlp = []

for objet in HUV_dataset['object']:

  HUV_dataset_objects_nlp.append(nlp(objet))

In [ ]:
# compute similarity    
objsimilarities = {}   
for word in listoftags:
    tok = nlp(word)
    objsimilarities[tok.text] ={}
    for tok_ in HUV_dataset_objects_nlp:

        #print(tok_)

        #tok_ = nlp(tok_)
        objsimilarities[tok.text].update({tok_.text:tok.similarity(tok_)})

# sort
objtop10 = lambda x: {k: v for k, v in sorted(objsimilarities[x].items(), key=lambda item: item[1], reverse=True)[:10]}

print('Done!')

In [ ]:
for tag in listoftags[:20]:

  print("Tag "+tag+ " close Words :")
  print(objtop10(tag))

Tag eating burger close Words :
{'eating apple': 0.7782340749061656, 'pizza cheese': 0.7524688047375979, 'cheese pizza': 0.7524688047375979, 'hamburger': 0.7506465414685103, 'sandwich': 0.7264423649902352, 'food': 0.7243706335953234, 'pizza': 0.7207653355035516, 'meat': 0.7173273142192711, 'steak': 0.7169213441225496, 'cheeseburger': 0.715374084132162}
Tag dining close Words :
{'dining table': 0.8943436908585392, 'outdoor furniture': 0.6757364930709249, 'outdoor grill': 0.6331220513976582, 'coffee table': 0.6147673353753536, 'fireplace': 0.6067397061272719, 'space bar': 0.6024288461258814, 'balcony': 0.5912305873157151, 'furniture': 0.5866282393818751, 'table': 0.543305178715405, 'kitchen appliance': 0.5397080159833199}
Tag eating hotdog close Words :
{'eating apple': 0.7355186471776982, 'hamburger': 0.7008172388508687, 'snack': 0.6717127558157777, 'pizza cheese': 0.6648871629200265, 'cheese pizza': 0.6648871629200265, 'meat': 0.664863347872518, 'pizza': 0.6579862617104214, 'cheeseburg